In [2]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF

from modAL.models import ActiveLearner

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
import math

%matplotlib inline

In [3]:
# cleaned version of the data after removing outliers and negative values
path='/workspace/malathi/steam_turbine_stage_1_POC/notebooks/version_1/df_sorted.csv'
data=pd.read_csv(path)
data.shape

(7077, 6)

In [4]:
INPUT_COLUMNS=[ 'expansion ratio','corrected speed','ImpulseStage_CurrentStroke']
OUTPUT_COLUMNS=['ImpulseStage_Power', 'Impulse_Discharge_Temperature', 'corrected mass flow']

In [32]:
X=data.drop(OUTPUT_COLUMNS,axis=1)
y=data['ImpulseStage_Power']

X=np.array(X)
y=np.array(y)

X_pool, X_test, y_pool, y_test = train_test_split(X,y, test_size=0.7, random_state=1)
#X_pool, X_test, y_pool, y_test = X_pool.reset_index(drop=True), X_test.reset_index(drop=True), y_pool.reset_index(drop=True), y_test.reset_index(drop=True)

# train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.8,random_state=4)

# print(train_X.shape,test_X.shape,train_y.shape,test_y.shape)

In [37]:
X_pool.shape

(2123, 3)

In [90]:
y_pool.shape

(2123,)

In [40]:
X_pool

array([[  7.40725773, 146.46291103,  10.59      ],
       [  1.3569408 , 136.96758849,  47.79      ],
       [  1.78699658, 140.52933031,  33.48      ],
       ...,
       [  2.11621504, 134.16301992,  30.62      ],
       [  7.26466999, 119.29813583,  10.59      ],
       [  1.6215268 , 141.73821773,  36.35      ]])

In [6]:
def GP_regression_std(regressor, data):
    _, std = regressor.predict(data, return_std=True)
    query_idx = np.argmax(std)
    return query_idx, data[query_idx]

In [8]:
#working trial1
# n_initial=100
# initial_idx = np.random.choice(range(len(X_pool)), size=n_initial, replace=False)
# X_training, y_training = X_pool.iloc[initial_idx], y_pool.iloc[initial_idx]


# kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3)) 
         
# regressor = ActiveLearner(
#     estimator=GaussianProcessRegressor(kernel=kernel),
#     query_strategy=GP_regression_std,
#     X_training=X_training.values.reshape(-1,3), y_training=y_training.values.reshape(-1,1)
# )
# y_pred, y_std = regressor.predict(X_test.values.reshape(-1,3), return_std=True)
# y_pred, y_std = y_pred, y_std

In [91]:
# trial-2 converted to numpy arrays -working
n_initial = 100
initial_idx = np.random.choice(range(len(X_pool)), size=n_initial, replace=False)
X_training, y_training = X_pool[initial_idx], y_pool[initial_idx]

kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3)) 
         

regressor = ActiveLearner(
    estimator=GaussianProcessRegressor(kernel=kernel),
    query_strategy=GP_regression_std,
    X_training=X_training.reshape(-1,3), y_training=y_training.reshape(-1,1)
)

In [86]:
y_training[0:5]

array([1112.0407715,  343.4978027,  928.5059204, 1219.0772705,
       1759.4953613])

In [85]:
X_training[0:5]

array([[  3.19794605, 135.09132015,  22.03      ],
       [  7.38173984, 141.30885981,  10.59      ],
       [  3.71547419, 137.59020561,  19.17      ],
       [  2.76868094, 129.19134173,  24.9       ],
       [  1.6459694 , 125.74765955,  36.35      ]])

In [87]:
X_training[0]

array([  3.19794605, 135.09132015,  22.03      ])

In [88]:
y_training[0]

1112.0407715

In [42]:
y_pred[0:10]

array([[1.00171902e+02],
       [5.87262782e-03],
       [9.64167088e+02],
       [5.32903511e+02],
       [2.22540000e+02],
       [7.62141438e+02],
       [1.10534304e+03],
       [1.11288494e+02],
       [4.52308448e-05],
       [2.01827941e+03]])

In [43]:
y_std[0:10]

array([0.95023409, 1.        , 0.29174304, 0.95988268, 0.9809069 ,
       0.76262184, 0.56872809, 0.99768268, 1.        , 0.00499566])

In [75]:
xx=X_pool.reshape(-1,3)

In [76]:
xx

array([[  7.40725773, 146.46291103,  10.59      ],
       [  1.3569408 , 136.96758849,  47.79      ],
       [  1.78699658, 140.52933031,  33.48      ],
       ...,
       [  2.11621504, 134.16301992,  30.62      ],
       [  7.26466999, 119.29813583,  10.59      ],
       [  1.6215268 , 141.73821773,  36.35      ]])

In [ ]:
xx[0]

In [ ]:
xx[0:5]

In [ ]:
X_pool[0:5]

In [ ]:
X_pool.loc[5947]

In [ ]:
query_idx, query_instance = regressor.query(X_pool.loc[5947].values.reshape(-1,3))
print(query_idx, query_instance)

In [ ]:
X_pool.iloc[0]

In [ ]:
y_pool[0:5]

In [18]:
X_pool=X_pool.values
y_pool=y_pool.values

In [21]:
yy=y_pool[0:5].copy()

In [22]:
yy=y_pool.reshape(-1,1)
yy[0:5]

array([[ 335.9832764],
       [1755.1898193],
       [1941.7845459],
       [ 725.1453247],
       [1583.4072266]])

In [62]:
a=y_pool[0].reshape(1,-1)
aa=a.flatten()
aa

array([335.9832764])

In [51]:
b=X_pool.reshape(-1,3)
b

array([[  7.40725773, 146.46291103,  10.59      ],
       [  1.3569408 , 136.96758849,  47.79      ],
       [  1.78699658, 140.52933031,  33.48      ],
       ...,
       [  2.11621504, 134.16301992,  30.62      ],
       [  7.26466999, 119.29813583,  10.59      ],
       [  1.6215268 , 141.73821773,  36.35      ]])

In [93]:
# with active learning
n_queries = 25
for idx in range(n_queries):
    query_idx, query_instance = regressor.query(X_pool)
    print(query_idx,'\t\t', query_instance)
    print('---------------------------')
    print(X_pool[query_idx],y_pool[query_idx])
    
    
   # regressor.teach(X_pool[query_idx].reshape(3,-1), y_pool[query_idx].reshape(1,-1))

0 		 [  7.40725773 146.46291103  10.59      ]
---------------------------
[  7.40725773 146.46291103  10.59      ] 335.9832764
0 		 [  7.40725773 146.46291103  10.59      ]
---------------------------
[  7.40725773 146.46291103  10.59      ] 335.9832764
0 		 [  7.40725773 146.46291103  10.59      ]
---------------------------
[  7.40725773 146.46291103  10.59      ] 335.9832764
0 		 [  7.40725773 146.46291103  10.59      ]
---------------------------
[  7.40725773 146.46291103  10.59      ] 335.9832764
0 		 [  7.40725773 146.46291103  10.59      ]
---------------------------
[  7.40725773 146.46291103  10.59      ] 335.9832764
0 		 [  7.40725773 146.46291103  10.59      ]
---------------------------
[  7.40725773 146.46291103  10.59      ] 335.9832764
0 		 [  7.40725773 146.46291103  10.59      ]
---------------------------
[  7.40725773 146.46291103  10.59      ] 335.9832764
0 		 [  7.40725773 146.46291103  10.59      ]
---------------------------
[  7.40725773 146.46291103  10.59    